In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.decomposition import NMF



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/swatishrimali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
train=pd.read_csv("labeledTrainData.tsv",header=0,delimiter='\t',quoting=3)
test=pd.read_csv("testData.tsv",header=0,delimiter='\t',quoting=3)

In [4]:
def review_to_words(raw_review):
    review_text=bs(raw_review).get_text()    #Removing HTML
    letters_only=re.sub("[^A-Za-z]"," ",review_text) # Removing non letters
    words=letters_only.lower().split() # Tokenizing 
    stops=set(stopwords.words('english'))
    no_stop_words =[w for w in words if w not in stops] # removing stop words
    return(" ".join(no_stop_words))            

In [ ]:
#for train
num_reviews=train['review'].size
cleaned_reviews_train=[]

for review in range(0,num_reviews):
    cleaned_reviews_train.append(review_to_words(train['review'][review]))
    

In [ ]:
#for test
num_reviews=test['review'].size
cleaned_reviews_test=[]

for review in range(0,num_reviews):
    cleaned_reviews_test.append(review_to_words(test['review'][review]))
    

In [ ]:
#for train
vectorizer=CountVectorizer(analyzer='word',max_features=25000)
train_data_review=vectorizer.fit_transform(cleaned_reviews_train)
train_data_review=train_data_review.toarray()
forest=RandomForestClassifier(n_estimators = 100)
forest=forest.fit(train_data_review,train['sentiment'])


In [ ]:
#for test
test_data_features = vectorizer.transform(cleaned_reviews_test)
test_data_features = test_data_features.toarray()
result = forest.predict(test_data_features)


In [ ]:
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Bag_of_Words_model.csv", index=False, quoting=3 )


In [ ]:
from sklearn.tree import export_graphviz

In [ ]:
model = RandomForestClassifier(n_estimators=10)
model.fit(train_data_review,train['sentiment'])
estimator = model.estimators_[5]


In [ ]:
from sklearn.externals.six import StringIO  

dot_data = StringIO()

In [ ]:
export_graphviz(estimator, out_file=dot_data, 
                feature_names = train['review'],
                class_names = train['sentiment'])